In [3]:
!pip install datasets openai tiktoken tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:0000:0100:01m


In [20]:
import os
import openai
from transformers.trainer_utils import set_seed
from datasets import load_dataset
import warnings
from pprint import pprint 
import random
import torch
from torch import Tensor
from transformers import BatchEncoding, pipeline
import math
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer
from transformers.utils import ModelOutput
import numpy as np
import faiss
from tqdm import tqdm

In [2]:
warnings.simplefilter('ignore')

# BPRの実装


In [22]:
# データの準備
train_dataset = load_dataset("llm-book/aio-retriever", split="train")

In [8]:
print(train_dataset)

Dataset({
    features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
    num_rows: 22335
})


In [13]:
pprint(train_dataset[1])

{'answers': ['骨川'],
 'competition': 'abc ～the first～',
 'negative_passage_indices': [0,
                              1,
                              2,
                              3,
                              4,
                              5,
                              6,
                              7,
                              8,
                              9,
                              10,
                              13,
                              14,
                              15,
                              17,
                              18,
                              19,
                              21,
                              22,
                              23,
                              24,
                              25,
                              26,
                              27,
                              28,
                              29,
                              30,
                              31,
   

In [23]:
# 正例と負例のパッセージを持たない事例を除去する
train_dataset = train_dataset.filter(lambda x: len(x['positive_passage_indices']) > 0 and len(x['negative_passage_indices']) > 0) 

In [24]:
# 訓練データセットの各事例の正例パッセージの先頭だけ残す（質問と最も関連度が高いため）

def filter_passages(example: dict) -> dict:
    """訓練セットの各事例で、正例のパッセージを最初の一つのみ残す"""
    example["positive_passage_indices"] = [example["positive_passage_indices"][0]]

    return example


train_dataset = train_dataset.map(filter_passages)

In [25]:
# datasetの確認
print(train_dataset)

Dataset({
    features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
    num_rows: 19596
})


In [27]:
# 検証データセットにも同様の処理を行う

valid_dataset = load_dataset("llm-book/aio-retriever", split="validation")
valid_dataset = valid_dataset.filter(lambda x: len(x['positive_passage_indices']) > 0 and len(x['negative_passage_indices']) > 0)

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [28]:
# 検証データでは正例パッセージと負例パッセージの先頭の事例のみを使用するようにする。
# ランダムで選ばれたものを使うと再現性がとれなくなるため？？

def filter_passages(example: dict) -> dict:
    """検証セットの各事例で、正例のパッセージと負例パッセージを最初の一つのみ残す"""
    example["positive_passage_indices"] = [example["positive_passage_indices"][0]]
    example["negative_passage_indices"] = [example["negative_passage_indices"][0]]

    return example

valid_dataset = valid_dataset.map(filter_passages)

Map:   0%|          | 0/864 [00:00<?, ? examples/s]

In [29]:
print(valid_dataset)

Dataset({
    features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
    num_rows: 864
})


### トークナイザとcollate関数

In [31]:
# tokenizer
base_model_name = "cl-tohoku/bert-base-japanese-v3"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/vocab.txt
loading file spiece.model from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/tokenizer_config.json
loading file tokenizer.json from cache at None


In [107]:
# collate
def collate_fn(examples: list[dict]) -> dict[str, BatchEncoding | Tensor]:
    """BPRの訓練・検証データのミニバッチを作成"""
    questions: list[str] = []
    passage_titles: list[str] = []
    passage_texts: list[str] = []

    for example in examples:
        questions.append(example['question'])

        # 事例の正例と負例を一つランダムで取得する
        positive_passage_idx = random.choice(example['positive_passage_indices'])
        negative_passage_idx = random.choice(example['negative_passage_indices'])


        # ピックアップされたidxのtitleとtextを格納
        passage_titles.extend(
            [
            example["passages"][positive_passage_idx]['title'],
            example["passages"][negative_passage_idx]['title']
            ]
        )
        
        passage_texts.extend([
            example["passages"][positive_passage_idx]['text'],
            example["passages"][negative_passage_idx]['text'],
        ])
        

    # 質問とパッセージをtokenize
    tokenized_question = tokenizer(
        questions,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt',
    )

    tokenized_passage = tokenizer(
        passage_titles,
        passage_texts,
        padding=True,
        truncation="only_second", # 2番目のペア（passage_texts）のみtrancateされる
        max_length=128,
        return_tensors='pt',
    )


     # 質問とパッセージのスコア行列における正例の位置を示すTensorを作成する
    # 行列の [0, 1, 2, ..., len(questions) - 1] 行目の事例（質問）に対して
    # [0, 2, 4, ..., 2 * (len(questions) - 1)] 列目の要素（パッセージ）が
    # 正例となる
    labels = torch.arange(0, 2 * len(questions), 2)
    return {
        "tokenized_questions": tokenized_question,
        "tokenized_passages": tokenized_passage,
        "labels": labels,
    }

### モデルの準備

In [153]:
class BPRModel(nn.Module):
    """BPRモデル"""
    def __init__(self, base_model_name: str):
        super().__init__()

        # エンコーダーの構築
        self.question_encoder = AutoModel.from_pretrained(base_model_name)
        self.passage_encoder = AutoModel.from_pretrained(base_model_name)

        # 訓練時のステップ数（バイナリの損失計算に使用）
        self.global_step = 0


    def binary_encode(self, x: Tensor) -> Tensor:
        """実数埋め込みをバイナリ変換"""
        if self.training:
            # 訓練時
            return torch.tanh(x * math.pow((1.0 + self.global_step * 0.1), 0.5))
        else:
            # 推論時
            return torch.where(x >= 0, 1.0, -1.0).to(x.device)


    def encode_questions(self, tokenized_questions: BatchEncoding) -> tuple[Tensor, Tensor]:
        """質問を実数埋め込みとバイナリに変換"""
        encoded_questions = self.question_encoder(**tokenized_questions).last_hidden_state[:, 0]
        binary_encoded_questions = self.binary_encode(encoded_questions)

        return encoded_questions, binary_encoded_questions
    
    def encode_passages(self, tokenized_passages: BatchEncoding) -> Tensor:
        """パッセージをバイナリに変換"""
        encoded_passages = self.passage_encoder(**tokenized_passages).last_hidden_state[:, 0]
        binary_encoded_passages = self.binary_encode(encoded_passages)
        
        return binary_encoded_passages


    def compute_loss(self, 
                     encoded_questions: Tensor, 
                     binary_encoded_questions: Tensor, 
                     binary_encoded_passages: Tensor, 
                     labels: Tensor,
                    )-> Tensor:
        
        """BPRの損失計算"""
        num_questions = encoded_questions.size(0)
        num_passages = binary_encoded_passages.size(0) # 正例と負例があるのでquestionより、2倍のバッチ数

        # 候補パッセージの計算
        # 質問のバイナリとパッセージのバイナリの内積をスコアに用いて
        # 正例パッセージのスコアと負例パッセージスコアのランキング損失を計算する。
        binary_scores = torch.matmul(binary_encoded_questions, binary_encoded_passages.transpose(0, 1)) # (バッチ数, 次元数)* (次元数, バッチ数(正例＋負例)) => (バッチ数、バッチ数*2)
        
        positive_binary_mask = F.one_hot(labels, num_classes=num_passages).bool() # 
        positive_binary_scores = torch.masked_select(binary_scores, positive_binary_mask).repeat_interleave(num_passages - 1) # 各question(行)に対応した正例のスコアがすべての列（num_passages - 1分）に反映される
        negative_binary_scores = torch.masked_select(binary_scores, ~positive_binary_mask) # num_passages - 1の列数になることに注意
        target =torch.ones_like( positive_binary_scores).long()
        loss_cand = F.margin_ranking_loss(positive_binary_scores,
                                          negative_binary_scores,
                                          target,
                                          margin=0.1
                                         )

        # 候補パッセージのリランキングの損失を計算する
        # 質問の実数埋め込みとパッセージのバイナリ埋め込みの内積を
        # スコアに用いて、正例パッセージのスコアと負例パッセージのスコアの
        # 交差エントロピー損失を計算する
        dense_scores = torch.matmul(encoded_questions, binary_encoded_passages.transpose(0, 1))
        loss_rerank = F.cross_entropy(dense_scores, labels)
        loss = loss_cand + loss_rerank
        return loss

    # 順伝搬
    def forward(self, tokenized_questions: BatchEncoding, tokenized_passages: BatchEncoding, labels: Tensor) -> ModelOutput:
        # questionとpassageを埋め込み
        encoded_questions, binary_encoded_questions = (self.encode_questions(tokenized_questions))
        binary_encoded_passages = (self.encode_passages(tokenized_passages))

        
        # BPRの計算
        loss = self.compute_loss(encoded_questions, binary_encoded_questions, binary_encoded_passages, labels)

        if self.training:
            self.global_step += 1

        return ModelOutput(loss=loss)


In [154]:
# modelの初期化
model = BPRModel(base_model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c7e/config.json
Model config BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-v3",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32768
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--cl-tohoku--bert-base-japanese-v3/snapshots/65243d6e5629b969c77309f217bd7b1a79d43c

In [155]:
# BPRの訓練のハイパーパラメータを設定する
training_args = TrainingArguments(
    output_dir="outputs_bpr",  # 結果の保存先フォルダ
    per_device_train_batch_size=32,  # 訓練時のバッチサイズ
    per_device_eval_batch_size=32,  # 評価時のバッチサイズ 
    learning_rate=1e-5,  # 学習率
    max_grad_norm=2.0,  # 勾配クリッピングにおけるノルムの最大値 ===========重要な追加点==============
    num_train_epochs=20,  # 訓練エポック数
    warmup_ratio=0.1,  # 学習率のウォームアップを行う長さ
    lr_scheduler_type="linear",  # 学習率のスケジューラの種類
    evaluation_strategy="epoch",  # 検証セットによる評価のタイミング
    logging_strategy="epoch",  # ロギングのタイミング
    save_strategy="epoch",  # チェックポイントの保存のタイミング
    save_total_limit=1,  # 保存するチェックポイントの最大数
    fp16=True,  # 自動混合精度演算の有効化
    remove_unused_columns=False,  # データセットの不要フィールドを削除するか
    report_to='all'
)

PyTorch: setting up devices


In [156]:
trainer = Trainer(model = model,
                  args=training_args,
                  data_collator=collate_fn,
                  train_dataset=train_dataset,
                  eval_dataset=valid_dataset
                 )

Using auto half precision backend


In [157]:
trainer.train()

***** Running training *****
  Num examples = 19,596
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12,260
  Number of trainable parameters = 222,414,336


Epoch,Training Loss,Validation Loss
1,4.434800,1.873021
2,0.716800,1.404888
3,0.506300,1.349858
4,0.366500,1.365851
5,0.296700,1.323755
6,0.248300,1.382280
7,0.208400,1.489815
8,0.176500,1.453172
9,0.167800,1.571055
10,0.151200,1.603697


***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
Saving model checkpoint to outputs_bpr/checkpoint-613
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
Saving model checkpoint to outputs_bpr/checkpoint-1226
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_bpr/checkpoint-613] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
Saving model checkpoint to outputs_bpr/checkpoint-1839
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_bpr/checkpoint-1226] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 864
  Batch size = 32
Saving model checkpoint to outputs_bpr/checkpoint-2452
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs_bpr/checkpoint-18

TrainOutput(global_step=12260, training_loss=0.4101210082335729, metrics={'train_runtime': 16519.3443, 'train_samples_per_second': 23.725, 'train_steps_per_second': 0.742, 'total_flos': 0.0, 'train_loss': 0.4101210082335729, 'epoch': 20.0})

In [158]:
# 質問エンコーダを保存
question_encoder_path = "outputs_bpr/question_encoder"
model.question_encoder.save_pretrained(question_encoder_path)
tokenizer.save_pretrained(question_encoder_path)

# パッセージエンコーダを保存
passage_encoder_path = "outputs_bpr/passage_encoder"
model.passage_encoder.save_pretrained(passage_encoder_path)
tokenizer.save_pretrained(passage_encoder_path)

Configuration saved in outputs_bpr/question_encoder/config.json
Model weights saved in outputs_bpr/question_encoder/model.safetensors
tokenizer config file saved in outputs_bpr/question_encoder/tokenizer_config.json
Special tokens file saved in outputs_bpr/question_encoder/special_tokens_map.json
Configuration saved in outputs_bpr/passage_encoder/config.json
Model weights saved in outputs_bpr/passage_encoder/model.safetensors
tokenizer config file saved in outputs_bpr/passage_encoder/tokenizer_config.json
Special tokens file saved in outputs_bpr/passage_encoder/special_tokens_map.json


('outputs_bpr/passage_encoder/tokenizer_config.json',
 'outputs_bpr/passage_encoder/special_tokens_map.json',
 'outputs_bpr/passage_encoder/vocab.txt',
 'outputs_bpr/passage_encoder/added_tokens.json')

In [103]:
a= collate_fn(train_dataset)

In [117]:

def test_collate_fn(examples: list[dict]) -> dict[str, BatchEncoding | Tensor]:
    """BPRの訓練・検証データのミニバッチを作成"""
    questions: list[str] = []
    passage_titles: list[str] = []
    passage_texts: list[str] = []
    count = 0

    for example in examples:
        questions.append(example['question'])

        # 事例の正例と負例を一つランダムで取得する
        positive_passage_idx = random.choice(example['positive_passage_indices'])
        negative_passage_idx = random.choice(example['negative_passage_indices'])


        # ピックアップされたidxのtitleとtextを格納
        passage_titles.extend(
            [
            example["passages"][positive_passage_idx]['title'],
            example["passages"][negative_passage_idx]['title']
            ]
        )
        
        passage_texts.extend([
            example["passages"][positive_passage_idx]['text'],
            example["passages"][negative_passage_idx]['text'],
        ])

        count += 1
        if count >= 2:
            break
        

    # 質問とパッセージをtokenize
    tokenized_question = tokenizer(
        questions,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors='pt',
    )

    tokenized_passage = tokenizer(
        passage_titles,
        passage_texts,
        padding=True,
        truncation="only_second", # 2番目のペア（passage_texts）のみtrancateされる
        max_length=128,
        return_tensors='pt',
    )


     # 質問とパッセージのスコア行列における正例の位置を示すTensorを作成する
    # 行列の [0, 1, 2, ..., len(questions) - 1] 行目の事例（質問）に対して
    # [0, 2, 4, ..., 2 * (len(questions) - 1)] 列目の要素（パッセージ）が
    # 正例となる
    labels = torch.arange(0, 2 * len(questions), 2)
    return {
        "tokenized_questions": tokenized_question,
        "tokenized_passages": tokenized_passage,
        "labels": labels,
        "questions": questions,
        "passage_titles": passage_titles
    }

In [118]:
a = test_collate_fn(train_dataset)

In [119]:
pprint(a)

{'labels': tensor([0, 2]),
 'passage_titles': ['ビヨンセ', 'クロエ・ベネット', 'ドラえもん', '剛田武'],
 'questions': ['「abc 〜the first〜」へようこそ!さて、ABC・・・と始まるアルファベットは、全部で何文字でしょう?',
               '人気漫画『ドラえもん』の登場人物で、ジャイアンの苗字は剛田ですが、スネ夫の苗字は何でしょう?'],
 'tokenized_passages': {'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

# BPRによるパッセージの埋め込み

In [3]:
# AI王データセットのパッセージデータを読み込む
passage_dataset = load_dataset("llm-book/aio-passages", split='train')

In [4]:
print(passage_dataset)

Dataset({
    features: ['id', 'pageid', 'revid', 'text', 'section', 'title'],
    num_rows: 4288198
})


In [5]:
model_path = "./outputs_bpr/passage_encoder/"

tokenizer = AutoTokenizer.from_pretrained(model_path)
passage_encoder = AutoModel.from_pretrained(model_path)

device = 'cuda:0'
passage_encoder.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32768, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### モデルによる埋め込み

In [6]:
def embed_passages(titles: list[str], texts: list[str]) -> np.ndarray:
    """BPRのパッセージエンコーダを用いてパッセージの埋め込みを計算"""
    # パッセージにトークナイザを適用
    tokenized_passages = tokenizer(
        titles,
        texts,
        padding=True,
        truncation="only_second",
        max_length=256,
        return_tensors="pt",
    ).to(device)

    # トークナイズされたパッセージを実数埋め込みに変換
    with torch.inference_mode():
        with torch.cuda.amp.autocast():
            encoded_passages = passage_encoder(
                **tokenized_passages
            ).last_hidden_state[:, 0]

    # 実数埋め込みをNumPyのarrayに変換
    emb = encoded_passages.cpu().numpy()
    # 0未満の値を0に、0以上の値を1に変換 -> バイナリに変換
    emb = np.where(emb < 0, 0, 1).astype(bool)
    # bool型からuint8型に変換
    emb = np.packbits(emb).reshape(emb.shape[0], -1)
    return emb
     

In [7]:
# パッセージデータのすべての事例に埋め込みを行う
passage_dataset = passage_dataset.map(
    lambda example: {
        "embeddings": list(embed_passages(example['title'], example['text']))
    },
    batched=True
)

Parameter 'function'=<function <lambda> at 0x7f7114bb5b40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/4288198 [00:00<?, ? examples/s]

KeyboardInterrupt: 

# 文書検索モデルとChatGPTを組み合わせる

In [13]:
dataset_name = "llm-book/aio-passages-bpr-bert-base-japanese-v3"
passage_dataset = load_dataset(dataset_name, split="train")

encoder_model_name = "llm-book/bert-base-japanese-v3-bpr-question-aio"
encoder_pipeline = pipeline(
    "feature-extraction", model=encoder_model_name, device="cuda:0"
)

Generating train split:   0%|          | 0/4288198 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [22]:
embed_size = encoder_pipeline.model.config.hidden_size
faiss_index = faiss.IndexBinaryIDMap2(faiss.IndexBinaryFlat(embed_size))

with tqdm(total = len(passage_dataset)) as pbar:
    i = 0
    for batch in passage_dataset.iter(batch_size=512):
        bs = len(batch['embeddings'])

        # 埋め込みuint8に変換
        batch_embeddings = np.array(batch['embeddings'], dtype=np.uint8)

        batch_indices = np.arange(i, i + bs, dtype=np.int64)

        # 埋め込みをインデックスに格納
        faiss_index.add_with_ids(batch_embeddings, batch_indices)


        pbar.update(n=bs)
        i += bs

100%|██████████| 4288198/4288198 [02:21<00:00, 30251.31it/s]


In [31]:
def embed_questions(questions: list[str]) -> np.ndarray:
    """質問文を実数埋め込みに変換"""
    output_tensor = encoder_pipeline(questions, return_tensors='pt')
    embeddings = np.vstack([t.squeeze(0)[0] for t in output_tensor]) # clsの埋め込みのみ取得
    return embeddings


def binarize_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """実数埋め込みをバイナリ埋め込みに変換"""
    binary_embeddings = np.where(embeddings < 0, 0, 1)
    # uint8型に変換
    packed_binary_embeddings = np.packbits(binary_embeddings, axis=-1)
    return packed_binary_embeddings

In [48]:
# 作成した関数のチェック
q_embed = embed_questions(["日本で一番高い山は何？"])
binary_q_embed = binarize_embeddings(q_embed)
print(f'実数埋め込みのshape: {q_embed.shape}')
print(f'バイナリ返還後のshape: {binary_q_embed.shape}')

実数埋め込みのshape: (1, 768)
バイナリ返還後のshape: (1, 96)


In [52]:
scores,  passage_ids = faiss_index.search(binary_q_embed, k=5)
print(passage_dataset[passage_ids[0]]["text"])

['日本の領土に占める山間部の割合はおよそ7割程度である。2020年現在、日本で一番高い山は富士山で、逆に一番低い山は日和山である。日本は新期造山帯に位置し、多くの火山が見られる。日本では昔から人々が山と共生する文化が培われてきた。それらの山は里山とよばれ、農村などでは山を共有地として村全体で管理し、薪をとったり土や山菜などを利用する目的で利用され、手入れをされてきた。しかし、明治時代になるとそれらの山の中には国有地にされてしまったものも多く存在する。また、日本の山の中には霊峰とよばれ、民間信仰の場にされた山も多くある。山の中には昔から金山や銀山として、近代では銅や石炭などを入手するため多くの鉱山が作られ、鉱毒や粉塵などが問題になった。特に近代において、山林の多くで木材を得るための大規模な伐採などが行われた。', '3000 m峰は、独立峰の富士山と御嶽山及び飛騨山脈(北アルプス)と赤石山脈(南アルプス)の山域に限られている。24番目に高い山は剱岳 (2999 m) で、3000 mにわずか1 m届いていない。標高3000 mを越える一帯は森林限界のハイマツ帯で、高山植物の群生地となっている。また多くの3000 m峰のハイマツ帯は、ライチョウ(雷鳥)の生息地となっている。', '三角点は一般的に眺望の利く場所に設置され一等三角点は970点余りに上るが、その中で風格のある山容、優れた眺望、高い知名度、さらに概ね標高1,000m以上で登りがいのある山が選定された。一等三角点の最高峰は、標高3,121 mの南アルプスの赤石岳である。三角点がその山の最高峰とは限らず、山頂に三角点より高い場所があり、その地点すなわち標高点や測定点を以て山の高さとしている例も少なくない(例:早池峰、御嶽山、金剛山)。また連山を形成している場合、最高峰でないピークに一等三角点が設置されている例もある(例:赤城山、穂高岳、阿蘇山)。特殊な例としては雲仙普賢岳のように、噴火により一等三角点が最高峰でなくなった事例もある。利尻岳中腹の長官山のように全く山頂とは離れた地点に一等三角点が設置されている例もある(山頂は二等三角点「利尻絶頂」)。', 'この項では世界最高峰と考えられていた山(せかいさいこうほうとかんがえられていたやま)について述べる。19世紀初頭までアンデスが世界で最も高い山脈だと考えられて

In [55]:
# faissのreconstructメソッドにidを渡すとそのpassageの埋め込みが取得できる
passage_id = 0
np.unpackbits(faiss_index.reconstruct(passage_id))

array([1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [83]:
def retrieve_passages(
    questions: list[str],
    binary_k: int = 2048,  # リランキングするための候補の取得数
    top_k: int = 3,  # リランキング後に出力する最終的なパッセージ数
) -> list[list[str]]:
    """質問から関連するパッセージを取得"""
    # 質問をバイナリベクトルに変換する
    q_embed = embed_questions(questions)
    q_bin_embed = binarize_embeddings(q_embed)

    # バイナリベクトルを使用して検索する -> 候補パッケージの取得
    # output -> shape(question数, 選出されたids:512)
    # 選出されたids:512は関連性が高い順に並んでいる => point(1)
    binary_k_scores, binary_k_p_ids = faiss_index.search(
        q_bin_embed, binary_k
    )

    batch_size = len(questions)
    embed_dim = q_bin_embed.shape[-1] * 8 # byte変換を行っているので＊８をおこなって、bit基準にしている
    # インデックスからパッセージのベクトルを復元
    p_uint8_embed = [
        faiss_index.reconstruct(int(p_id))
        for p_id in binary_k_p_ids.flatten() # 1次元にreshape
    ]
    # uint8からboolに変換
    p_bin_embed = np.vstack([np.unpackbits(e) for e in p_uint8_embed])
    # 型とシェイプを変換
    p_bin_embed = p_bin_embed.astype(np.float32).reshape(
        batch_size, binary_k, embed_dim
    )
    p_bin_embed = p_bin_embed * 2 - 1 # 0, 1のバイナリ値を-1, 1のバイナリに変換している

    # 質問の実数ベクトルとパッセージのバイナリベクトルで再度スコアを計算する
    re_scores = np.einsum("ijk,ik->ij", p_bin_embed, q_embed)
    top_k_indices = np.argsort(-re_scores, axis=-1)[:, :top_k] 
    top_k_p_ids = np.take_along_axis(
        binary_k_p_ids, top_k_indices, axis=-1
    )

    # top_kのテキストを整形して出力する
    retrieved_texts: list[list[str]] = []
    for p_ids in top_k_p_ids:
        formatted_texts = [
            f"タイトル：{passage_dataset[i]['title']}\n"
            f"本文：{passage_dataset[i]['text']}"
            for i in p_ids.tolist()
        ]
        retrieved_texts.append(formatted_texts)

    return retrieved_texts

In [84]:

passages = retrieve_passages(["日本で一番高い山は何？"], top_k=3)[0]
for passage in passages:
    print(passage)
     

タイトル：日本の山
本文：日本の領土に占める山間部の割合はおよそ7割程度である。2020年現在、日本で一番高い山は富士山で、逆に一番低い山は日和山である。日本は新期造山帯に位置し、多くの火山が見られる。日本では昔から人々が山と共生する文化が培われてきた。それらの山は里山とよばれ、農村などでは山を共有地として村全体で管理し、薪をとったり土や山菜などを利用する目的で利用され、手入れをされてきた。しかし、明治時代になるとそれらの山の中には国有地にされてしまったものも多く存在する。また、日本の山の中には霊峰とよばれ、民間信仰の場にされた山も多くある。山の中には昔から金山や銀山として、近代では銅や石炭などを入手するため多くの鉱山が作られ、鉱毒や粉塵などが問題になった。特に近代において、山林の多くで木材を得るための大規模な伐採などが行われた。
タイトル：日本の山一覧 (3000m峰)
本文：3000 m峰は、独立峰の富士山と御嶽山及び飛騨山脈(北アルプス)と赤石山脈(南アルプス)の山域に限られている。24番目に高い山は剱岳 (2999 m) で、3000 mにわずか1 m届いていない。標高3000 mを越える一帯は森林限界のハイマツ帯で、高山植物の群生地となっている。また多くの3000 m峰のハイマツ帯は、ライチョウ(雷鳥)の生息地となっている。
タイトル：一等三角点百名山
本文：三角点は一般的に眺望の利く場所に設置され一等三角点は970点余りに上るが、その中で風格のある山容、優れた眺望、高い知名度、さらに概ね標高1,000m以上で登りがいのある山が選定された。一等三角点の最高峰は、標高3,121 mの南アルプスの赤石岳である。三角点がその山の最高峰とは限らず、山頂に三角点より高い場所があり、その地点すなわち標高点や測定点を以て山の高さとしている例も少なくない(例:早池峰、御嶽山、金剛山)。また連山を形成している場合、最高峰でないピークに一等三角点が設置されている例もある(例:赤城山、穂高岳、阿蘇山)。特殊な例としては雲仙普賢岳のように、噴火により一等三角点が最高峰でなくなった事例もある。利尻岳中腹の長官山のように全く山頂とは離れた地点に一等三角点が設置されている例もある(山頂は二等三角点「利尻絶頂」)。


In [67]:
q_embed = embed_questions(["日本で一番高い山は何？", "世界で一番高い山は？"])
q_bin_embed = binarize_embeddings(q_embed) 

In [68]:
q_bin_embed.shape

(2, 96)

In [70]:
binary_k_scores, binary_k_p_ids = faiss_index.search(
    q_bin_embed, 512
)

In [74]:
binary_k_p_ids.shape

(2, 512)

In [75]:
binary_k_p_ids.flatten().shape

(1024,)

In [82]:
re_scores = np.array([
    [1, 2, 3, 4, 5, 6, 7],
    [7, 6, 5, 4, 3, 2, 1]
])

top_k = 2

top_k_indices = np.argsort(-re_scores, axis=-1)[:, :top_k]
print(top_k_indices)

[[6 5]
 [0 1]]
